# DataFrame

## Introduction



# Import some helper codes

* fs - file systems utitlities (part of nodejs)
* -init.js _is a bunch of helper code_

In [1]:
var fs = require('fs');
var request = require('request');
var _init = fs.readFileSync('init.js').toString();
eval(_init);
var {frameWithIndex} = DataFrame;
//setupD3();
//var fr = require("../lib")

'LOADED'

In [ ]:
DataFrame.frameWithIndex

# Read some covid related data

In [2]:

var covid, continents;
var WORLD_DATA_SRC ='https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv';

var r = request.get(WORLD_DATA_SRC, function (error, response, body) {
    if (!error && response.statusCode == 200) {
        var csv = body;
        covid = frameFromBuffer(csv.toString(),csvLine);
        //frame.data = frame.data.slice(0, frame.length-2)
    }
});





undefined

# Show a sample of the data

Take to first 20

In [7]:
// Display  first 20 rows the data frame just read
var x =covid.slice(1000,1010);
console.log(JSON.stringify(x.asObj(2),undefined, ' '))

{
 "__data": [
  "AGO",
  "Africa",
  "Angola",
  "2020-07-19",
  687,
  49,
  29.143,
  29,
  0,
  0.571,
  20.903,
  1.491,
  0.887,
  0.882,
  0,
  0.017,
  "",
  "",
  "",
  "",
  "",
  "",
  "",
  "",
  "",
  76.39,
  32866268,
  23.89,
  16.8,
  2.405,
  1.362,
  5819.495,
  "",
  276.045,
  3.94,
  "",
  "",
  26.664,
  "",
  61.15,
  0.581
 ],
 "__index$": 2
}


undefined

In [19]:
var y =covid.select(['iso_code', 'location', 'continent', 
              'population', 'diabetes_prevalence',
              'gdp_per_capita'
             ],ro => ro.continent === 'Europe').distinct();
frameWithIndex(y).filter(ro => ro.__index$ < 5) 

Ix,INDEX,iso code,location,continent,population,diabetes prevalence,gdp per capita
0,0,ALB,Albania,Europe,2877800,10.08,11803.431
1,1,AND,Andorra,Europe,77265,7.97,
2,2,AUT,Austria,Europe,9006400,6.35,45436.686
3,3,BEL,Belgium,Europe,11589616,4.29,42658.576
4,4,BGR,Bulgaria,Europe,6948445,5.81,18563.307


# print the number of rows and columns names

In [ ]:
({ rows: covid.length, columns: covid.columns})
function isFunc(f) { return typeof f === 'function'; }

## Keep only the columns we are interesed in

In [ ]:
function UPPER(x) { return x.toUpperCase(); }
console.log(typeof UPPER)
function asNum(x) {
    if(!x) return 0;
    return +x;
}
//covid = covid.convertData();
var frame = covid.select([
  ['iso_code', x => x || '---'],
  'continent',
  ['location=country', UPPER], // rename location to country
  'date',
  'total_cases',
  'new_cases',
  'total_deaths',
  'new_deaths',
  'new_tests',
  'total_tests',
  'positive_rate',
  'population',
  ['dummy', (x,ro,ix) => ix+1]
], (ro)=> ro.total_cases > 10  );

// show first 20
frame.groupBy(['iso_code','country', gb.max('total_cases')])
    .groupBy(['iso_code', gb.count('iso_code', 'count')]).sort(['count']) 
    //.filter(ro => ro.total_cases < 100)

In [ ]:
var x = covid.slice(100, 110);
x

## Sample 1% (0.001) of the data and show the first 20

In [ ]:
frame.filter(()=> Math.random() < 0.01 ) // take random 1% of data
    .slice(0,20)                          // take the first 20 
    .sort(['continent', 'country'])      // sort by contiment and country

In [ ]:
// Since all data is stored as strings
// The round function will conver it to a numeric round the value
// convert back to a string


function round(v) {
    var r = +v;
    if(isNaN(r)) return '0';
    return Math.round(r).toString();
}
round('abc')

In [ ]:
frameNumericColumns(frame)

In [ ]:

var ALL_COLUMNS = null;
function forceZero(v) { return round(v || '0')}
var TO_ZERO = Object.fromEntries(arrProd(frameNumericColumns(frame),forceZero));
frame = frame.update(TO_ZERO)
frame

## Print the number rows and max total_cases by country

In [ ]:


var round_some_columns =  { 
    total_cases: round, 
    max_new_cases_in_a_day: round,
    avg_new_cases_in_a_day: round
};

frame.groupBy(['continent','country', // columns to group on
               gb.count('date', 'rows'), // count the number of values in the date column after grouping
               gb.max('total_cases'), // get the max value in the total_cases column for the group 
               gb.max('new_cases', 'max_new_cases_in_a_day'), // get the max value in the total_cases 
                                                             // column for the group and give it the name 'max_new_cases_in_a_day 
               gb.mean('new_cases', 'avg_new_cases_in_a_day') // get the average value for new cases
              ])
    .update( {...round_some_columns, avg_new_cases_in_a_day: (v,ro) => round(ro.total_cases/ro.rows)} )
    .sort(['-rows', '-continent','-total_cases']) // rows, continent and total_cases is sorted in descending order: 

In [ ]:
console.log(22/161.0);
frame//.filter(r => r.country === "Laos")
    .groupBy(['continent','country', // columns to group on
               gb.count('date', 'rows'), // count the number of values in the date column after grouping
               gb.max('total_cases'), // get the max value in the total_cases column for the group 
               gb.max('new_cases', 'max_new_cases_in_a_day'), // get the max value in the total_cases 
                                                             // column for the group and give it the name 'max_new_cases_in_a_day 
               gb.mean('new_cases', 'avg_new_cases_in_a_day'), // get the average value for new cases
               gb.sum('new_cases', 'total_new_cases') // get the average value for new cases
             ])


In [ ]:
var CONTINENTS = "https://pkgstore.datahub.io/JohnSnowLabs/country-and-continent-codes-list/country-and-continent-codes-list-csv_csv/data/b7876b7f496677669644f3d1069d3121/country-and-continent-codes-list-csv_csv.csv"

request.get(CONTINENTS, function (error, response, body) {
    if (!error && response.statusCode == 200) {
        var csv = body;
        continents = frameFromBuffer(csv.toString(),'',csvLine);
        //frame.data = frame.data.slice(0, frame.length-2)
    }
});


In [ ]:
continents

In [ ]:
var contMap = {
  Brunei: 'Asia',
  Curacao: 'North America',
  'Democratic Republic of Congo': 'Africa',
  'Faeroe Islands': 'Europe',
  International: 'Asia',
  Palestine: 'Asia',
  Russia: 'Europe',
  'Saint Barthlemy': 'North America',
  'Saint Martin (French part)': 'North America',
  'South Korea': 'Asia',
  'United Kingdom': 'Europe',
  'United States': 'North America',
  Vatican: 'Europe',
  World: 'World'
};

In [ ]:
var code = fs.readFileSync('../../nurulc.github.io/javascripts/barChart.js').toString();
console.log(code);
var pre= ('<script>'+code+'</script><div id="aaa">AAA</div>');
//console.log(code)
function showC(aFrame) {
    let id = genID();
    let s =  JSON.stringify(aFrame);
    console.log(s)
    $$.html(`${pre}<div id="${id}">aaa</div><script>
    frameBarChart(${s}, "${id}",{
    width: 800, height: 600, key: "continent", value: "total_cases"
     })
    </script>`)
}
showC(disp);


In [ ]:
var aFrame = disp
var numerics = frameNumericColumns(aFrame);
var nonNums = arrRemove(aFrame.columns, numerics);
var key = nonNums[0];
var value = numerics[0];
({numerics, nonNums, key, value})

In [ ]:
frameChart(disp, {
    key: 'continent', title: "Coronavirus infections", 
    width: 850, axisX: 'Continents', axisY:"Infections"})
)

In [ ]:
var _init = fs.readFileSync('init.js').toString();
eval(_init);
setupD3();

In [ ]:
frameBarChart(disp, {
     title: "Coronavirus infections",  axisX: 'Continents', axisY:"Infections", isLog: false})

In [ ]:
continents.columns

In [ ]:
var WORLD_DATA_SRC = "https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide.csv"  ;
var frame, continents;
var CONTINENTS = "https://pkgstore.datahub.io/JohnSnowLabs/country-and-continent-codes-list/country-and-continent-codes-list-csv_csv/data/b7876b7f496677669644f3d1069d3121/country-and-continent-codes-list-csv_csv.csv"
function getData(src,result,ix)
    request.get(src, function (error, response, body) {
        if (!error && response.statusCode == 200) {
            var csv = body;
            result[ix] = frameFromBuffer(csv.toString(),'',csvLine);
            //frame.data = frame.data.slice(0, frame.length-2)
        }
    });
}
var result = [null,null]
getData(WORLD_DATA_SRC,result,0);
getData(CONTINENTS,result,1);

In [ ]:
function organizeData(frame, continents) {
        const ZERO = () => '0';
        const DATE = colName => r => r[colName].replace(/(\d\d)\/(\d\d)\/(\d\d\d\d)/,"$3-$2-$1");
        const Country_Name =  l => l === 'CANADA'?'Canada': l;
        const others = new Frame(
                    [
                      [
                        'Asia',
                        'Cases on an international conveyance Japan',
                        'JPG11668'
                      ],
                      [ 'Ocenia', 'French Polynesia', 'PYF' ],
                      [ 'Europe', 'Greece', 'EL' ],
                      [ 'Europe', 'Kosovo', 'XK' ],
                      [ 'Europe', 'United Kingdom', 'UK' ]
                    ],
                    ['Continent_Name', 'Country_Name', 'Two_Letter_Country_Code' ]);
  
        let conts = continents.groupBy(['Continent_Name', 'Country_Name', 'Two_Letter_Country_Code' ])
                              .concat(others)
                              .groupBy(['Country_Name', 'Two_Letter_Country_Code', 
                                         gb.min('Continent_Name', 'continent')]);
       
        function toNum(v) {
          const nv = +v;
          if (isNaN(nv)) {
            return 0;
          }
          return nv;
        }

        function accum(group, name) {
            let accum = 0, groupName=undefined;
            return (r => {
                if(r[group] !== groupName) {
                    accum = 0;
                    groupName = r[group]
                }
                let val = toNum(r[name]);
                return (accum += val);
            })
        } 
//         let conts = continents.groupBy(['Country_Name', 'Two_Letter_Country_Code', gb.min('Continent_Name', 'continent')]);
//        console.log(conts.columns)
        return frame.project(undefined, {"Countries and territories": Country_Name}).sort(['Countries and territories', 'Year', 'Month', 'Day'])
            .project([
                  'DateRep=date', 'Cases=new_cases',
                  'Deaths=new_deaths', 'Countries and territories=location',
                  'GeoId', 'total_cases', 'total_deaths'
                ], 
                {total_cases: accum('GeoId', 'Cases'), 
                 total_deaths: accum('GeoId', 'Deaths'),
                 date: DATE('DateRep'),
                 location: r => r['Countries and territories'].replace(/_/g, ' ')
                }, 
                true
            ).leftJoin(conts, ['continent', 'location', 'date', 'new_cases' ,
                        'new_deaths', 'total_cases', 'total_deaths', 'GeoId=tlc'
                       ], 'GeoId==Two_Letter_Country_Code')
            
}
organizeData(frame,continents)
    //.groupBy(['continent', 'location', 'tlc'])
    .filter(r => r.location === 'CANADA' || r.location === 'Canada')
//'03/03/2020'.replace(/(\d\d)\/(\d\d)\/(\d\d\d\d)/,"$3-$2-$1");
//'03/03/2020'.match(/(\d\d)\/(\d\d)\/(\d\d\d\d)/)

//organizeData(frame,continents).groupBy(['continent', 'location', 'tlc']).filter(r => !r.continent).data

In [ ]:
frame.sort(['GeoId', 'Year', 'Month', 'Day'])
     .groupBy(['Countries and territories', 'GeoId', 
               gb.sum('Cases'), gb.sum('Deaths')])
     .project(['Countries and territories=Location', 'GeoId', 'Cases', 'Deaths'])
     
                                                       

In [ ]:
cont

In [ ]:
frameBarChart(disp, {
     title: "Coronavirus deaths",  value: 'total_deaths', axisX: 'Continents', axisY:"Deaths", isLog: false})

In [ ]:
frameNumericColumns(disp);


In [ ]:
//var {Frame, frameFromBuffer, csvLine, tsvLine, gb} = exports;
frame.groupBy(['country',gb.count('date','count'), 
               gb.max('total_cases'), 
               gb.max('total_deaths')]
             ).sort(['-total_cases'])


In [ ]:
var s = readZipData('../data/reference/all_icd10_codes.tsv.gz').toString('utf8');
var {Frame, frameFromBuffer, tsvLine} = exports;
var frame = frameFromBuffer(s,'',tsvLine);
//frame
frame.groupBy(['TYPE'])

In [ ]:
var ff = require('../lib');
//var Frame = ff.Frame;

In [ ]:
function doN(n, fn) {
    return () =>{
        var res;
        for(let i=0;i<n; i++) { res = fn(res); }
        return res;
    }
}

In [ ]:
var ICD10 = readZipFrame('../data/reference/all_icd10_codes.tsv.gz');
var A = r => r.ICD10[0] === 'A';
ICD10.slice(0,20);

In [ ]:
 var _ = timeOf(doN(1, () => ICD10.data.filter(r => r[0][0] === 'A' )));


In [ ]:
function filter(fr,fn) {
    let data = fr.data;
    let res = [];
    for(let i=0; i<data.length; i++) {
        let d = data[i];
        if(fn(d,i)) res.push(d);
    }
    return res;
}
var _ = timeOf(doN(1, () => filter(ICD10,r => r[0][0] === 'A' )));


ICD10.groupBy(['TYPE'])

In [ ]:
var exp = require('../dist/bundle');
exp

In [ ]:
function getAllMethods(obj) {
    let props = []

    do {
        const l = Object.getOwnPropertyNames(obj)
            .concat(Object.getOwnPropertySymbols(obj).map(s => s.toString()))
            .sort()
            .filter((p, i, arr) =>
                typeof obj[p] === 'function' &&  //only the methods
                p !== 'constructor' &&           //not the constructor
                (i == 0 || p !== arr[i - 1]) &&  //not overriding in this prototype
                props.indexOf(p) === -1          //not overridden in a child
            )
        props = props.concat(l)
    }
    while (
        (obj = Object.getPrototypeOf(obj)) &&   //walk-up the prototype chain
        Object.getPrototypeOf(obj)              //not the the Object prototype methods (hasOwnProperty, etc...)
    )

    return props
}
getAllMethods( new exp.Frame());


In [ ]:
var exp = require('../lib');
function PICK(name,...args) {
    return o => o[name](...args);
}
Object.keys(exp).filter(s => s.match(/__(.*)__/))
    .map(s => s.replace(/__(.*)__/, "$1"))
    .map(PICK('toLowerCase')).map(n => require('../lib/'+n));


In [ ]:
var CNT = 1
var myData = timeOf(doN(CNT, () =>ICD10.filter(r => !r.ICD10.match(/^A/) ).project(['ICD10'])))
var __ =   timeOf(doN(CNT, () =>ICD10.filter(r => !r.ICD10.startsWith('A') ).project(['ICD10'])))
var ___ =  timeOf(doN(CNT, () =>ICD10.filter(NOT(A)).project(['ICD10'])))


__

In [ ]:
var getMethods = (obj) => Object.getOwnPropertyNames(obj).filter(item => typeof obj[item] === 'function')
CNT=1;
console.log('len', ICD10.length)
var x =timeOf(doN(CNT,() => ICD10.innerJoin(myData, [...ICD10.columns.map(x => '1.'+x)], 'ICD10')))//.slice(10,20)
timeOf(() =>ICD10.project(["T", 'TYPE', 'ICD10'], {T: r => r.ICD10[0] },true)
    .groupBy(['T', 'TYPE', gb.count('ICD10', 'CNT')])
    .sort(['-CNT']));
//gb

In [ ]:
function renameColumns(frame,map) {
	return frame.columns.map(c => map[c]?map[c]:c);

}



In [ ]:
var NAMES = new Frame([['PCS', "Procedure"], ['CM', "Diagnostic"],["???", "Unknown"]], ["TYPE", "DESC"], "ICD-TYPE");
var VALUES = new Frame([['PCS', "Institution"], ['CM', "Any"],["UNK", "Unknown"]], ["TYPE", "LOC"], "ICD-TYPE");



VALUES

In [ ]:
NAMES

In [ ]:
NAMES.leftJoin(ICD10, renameColumns(ICD10, {TYPE: '1.DESC=TYPE'}), "TYPE")
      .groupBy(['TYPE', gb.count('TYPE','CNT')])//.groupBy([gb.min('CNT')])

In [ ]:
NAMES.leftJoin(VALUES, "1.TYPE,DESC,LOC".split(','), 'TYPE')

In [ ]:
NAMES.outerJoin(VALUES, "1.TYPE,DESC,LOC".split(','), 'TYPE').sort(["DESC", "TYPE"])

In [ ]:
NAMES.innerJoin(ICD10, renameColumns(ICD10, {TYPE: '1.DESC'}), "TYPE")

In [ ]:
NAMES.outerJoin(ICD10, renameColumns(ICD10, {TYPE: '1.DESC'}), "TYPE")

In [ ]:
//var {Frame} = require('../dist/bundle.js');

In [ ]:
var C = class {
    constructor(a) {
        this.value = a;
    }
    
    
};

var c = new C(1);
new c.constructor(2);
function timeOf(fn) {
  const name = 'func';
  const t0 = new Date().getTime();

  const res = fn();

  const t1 = new Date().getTime();
  console.log(`Call to ${name} took ${t1 - t0} milliseconds.`);
  return res;
}

In [ ]:
var la = require('../lib/array')

In [ ]:
function test(n) {
    var a = la.newArray(n);
    for(let i=0; i<n; i++) a[i] = i;
    return a;
}

function test0(n) {
    var a = new Int32Array(n);
    for(let i=0; i<n; i++) a[i] = i;
    return a;
}

function test2(n) {
    var a = [];
    for(let i=0; i<n; i++) a.push(i);
    return a;
}

function timeOf(fn) {
  const name = 'func';
  const t0 = new Date().getTime();

  const res = fn();

  const t1 = new Date().getTime();
  console.log(`Call to ${name} took ${t1 - t0} milliseconds.`);
  return res;
}

[timeOf(() => la.arrSum(test(10000000))),
timeOf(() => la.arrSum(test0(10000000)))];
var a = test0(10000000);
var b = test2(10000000);


In [ ]:
[timeOf(() => {for(i=0; i<100; i++)la.arrSum(b)}),
timeOf(() => {for(i=0; i<100; i++)la.arrSum(a)})]

In [ ]:
//var {Frame} = require('../lib/frame')

In [ ]:
var f = new Frame( [[1,2],[3,4]],['a','b'],'test');
ArrayBuffer.isView(new Int32Array(10))

In [ ]:
f.sort(['-b'])

In [ ]:
fs.readdirSync('../data/reference');
var ICD9 = readZipFrame('../data/reference/all_icd10_codes.tsv.gz');
var HCPCS = readZipFrame('../data/reference/hcpcs.tsv.gz')
HCPCS

In [ ]:
HCPCS = HCPCS.project(['TYPE', ...HCPCS.columns], {TYPE: r => r.CODE.substr(0,2)},true);
HCPCS.groupBy(['TYPE', gb.count('TYPE', 'COUNT')]).sort(['-COUNT'])

In [ ]:
HCPCS.filter(r => r.TYPE === '90').groupBy(['TYPE', gb.count('TYPE', 'COUNT')])


In [ ]:
$$.html(`<script>
console.log(define.toString())
</script>`)


In [ ]:
var CFrame = class {
    constructor(data, columns, isColumn=true) {
        this.columns = columns;
        let len = isColumn? data[0].length : data.length;
        this.length = len;
        if(!column) {
           let rLn = this.columns.length;
           let aData = arrayOf(() => new Array(len), rLn);
           this.data = aData;
           for(let i=0; i<len; i++) {
               let row = data[i];
               let j = 0;
               for(; j< rLn-7; j+= 8) {
                   aData[j][i] = row[j];
                   aData[j+1][i] = row[j+1];
                   aData[j+2][i] = row[j+2];
                   aData[j+3][i] = row[j+3];
                   aData[j+4][i] = row[j+4];
                   aData[j+5][i] = row[j+5];
                   aData[j+6][i] = row[j+6];
                   aData[j+7][i] = row[j+7];
               }
               for(; j< rLn-3; j+= 4) {
                   aData[j][i] = row[j];
                   aData[j+1][i] = row[j+1];
                   aData[j+2][i] = row[j+2];
                   aData[j+3][i] = row[j+3];
               }
               for(; j< rLn; j++) {
                   aData[j][i] = row[j]
               }
           }
       }
    }

    /**
     * old Column array
    _colMapper(mapArray, baseData) {
        
    }*/
}

#Adam optimizer

In [ ]:
var result, result1;
var http = require('https');
var res = http.get("https://raw.githubusercontent.com/cmusam/fortune500/master/csv/fortune500-2017.csv",
         (resp) =>{ 
  let data = '';

  // A chunk of data has been recieved.
  resp.on('data', (chunk) => {
    data += chunk;
  });

  // The whole response has been received. Print out the result.
  resp.on('end', () => {
    result1 = data.replace(/\r/g, '');
    result =  result1 .split('\n').map(s => s.split(','));
  });
});


In [ ]:
var fortune = timeOf(() => new Frame(result.slice(1),result[0],'Fortune')
                     .sort(["-profit ($ millions)"]))
//fortune.filter(r => console.log(r.__data[1]))
//fortune.filterRaw(r => r[1] === undefined)
fortune

In [ ]:
//fortune.filter(r => { console.log(r, r.company[0]); return true; })
fortune.filter(r => r.company !== undefined )

In [ ]:
var fortune = timeOf(() => ff.frameFromBuffer(result1,'fortune',ff.csvLine).sort(["-profit ($ millions)"]))
fortune.filter(r => {  var n = r.company;  console.log(r.company); return n.substr(0,1) === 'A'})


In [ ]:
Object.fromEntries(Object.entries({a:1,b:2,c:3}))

In [ ]:
console.log(JSON.stringify({a:'1',b:2, c:new Date()},null,' '))